<a href="https://colab.research.google.com/github/asmamest/tp_bigdata/blob/main/compte_rendu_Asma_Mestaysser_3InfoA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Compte Rendu TP Spark - Asma Mestaysser

In [7]:
# Installation de PySpark
!pip install pyspark

# Import des bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import lit
from google.colab import files
import pandas as pd


In [8]:
# 1- Créer une SparkSession et la démarrer
spark = SparkSession.builder \
    .appName("Système de Recommandation") \
    .getOrCreate()

# Chargement du fichier
uploaded = files.upload()

Saving rating2.csv to rating2 (2).csv


In [9]:
# 3- Charger les données de « rating2.csv » dans un dataframe df
filename = list(uploaded.keys())[0]
df = spark.read.csv(filename, header=True, inferSchema=True)

# 4- Afficher le contenu de df
print("Contenu du DataFrame :")
df.show(10)
print("Schéma du DataFrame :")
df.printSchema()


Contenu du DataFrame :
+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
|     1|    112|   3.5|1094785740|
|     1|    151|   4.0|1094785734|
|     1|    223|   4.0|1112485573|
|     1|    253|   4.0|1112484940|
|     1|    260|   4.0|1112484826|
+------+-------+------+----------+
only showing top 10 rows

Schéma du DataFrame :
root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [10]:

# 5- Supprimer les colonnes inutilisables pour cet exercice
columns_to_keep = ['userId', 'movieId', 'rating']
existing_columns = df.columns
columns_to_drop = [col for col in existing_columns if col not in columns_to_keep]

if columns_to_drop:
    df = df.drop(*columns_to_drop)
    print(f"Colonnes supprimées : {columns_to_drop}")

print("DataFrame après suppression des colonnes inutiles :")
df.show(10)

# Vérifier les valeurs nulles
print("Valeurs nulles par colonne :")
from pyspark.sql.functions import col, sum as spark_sum
df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

# Nettoyer les données
df_clean = df.na.drop()
print(f"Nombre de lignes avant nettoyage : {df.count()}")
print(f"Nombre de lignes après nettoyage : {df_clean.count()}")

Colonnes supprimées : ['timestamp']
DataFrame après suppression des colonnes inutiles :
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
+------+-------+------+
only showing top 10 rows

Valeurs nulles par colonne :
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     0|      0|     0|
+------+-------+------+

Nombre de lignes avant nettoyage : 999999
Nombre de lignes après nettoyage : 999999


In [11]:
# 6- Diviser le dataset en training et testing sets (0.8, 0.2)
training, test = df_clean.randomSplit([0.8, 0.2], seed=42)
print(f"Training set: {training.count()} lignes")
print(f"Test set: {test.count()} lignes")


Training set: 799967 lignes
Test set: 200032 lignes


In [12]:
# 7- Créer le modèle ALS
als = ALS(
    maxIter=5,
    regParam=0.01,
    userCol='userId',
    itemCol='movieId',
    ratingCol='rating',
    coldStartStrategy="drop"  # Pour éviter les erreurs avec des utilisateurs/items non vus
)

"""
- maxIter : nombre maximum d'itérations que l'algorithme ALS va exécuter
- regParam : paramètre de régularisation pour éviter le surapprentissage (overfitting)
"""

# 8- Créer le modèle en utilisant als.fit(training)
model = als.fit(training)

# 9- Créer une variable pred pour tester les performances du modèle
predictions = model.transform(test)

# 10- Analyser le résultat obtenu « pred »
print("Prédictions sur le jeu de test :")
predictions.show(10)

Prédictions sur le jeu de test :
+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   471|   1088|   2.0| 3.8851678|
|   471|   1959|   4.0| 3.8692205|
|  1829|   1580|   3.5| 4.5470896|
|  1959|   1580|   3.0| 3.1491845|
|  2866|   1580|   3.0| 3.1816525|
|  3749|   1088|   3.0| 3.6177936|
|  1395|    471|   3.0| 2.7293782|
|  1460|    471|   5.0|  4.656977|
|  1507|   1591|   1.5| 2.4445825|
|  2387|   8638|   4.0| 3.1791344|
+------+-------+------+----------+
only showing top 10 rows



In [13]:
# 11- Calculer le « rmse » entre les colonnes pred et rating
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"RMSE = {rmse}")

RMSE = 0.8914683069616569


In [14]:
# 12- Afficher le user 600 et les différents ids des films votés par ce user
user_600_ratings = df_clean.filter(df_clean.userId == 600)
print("Films notés par l'utilisateur 600 :")
user_600_ratings.show()

Films notés par l'utilisateur 600 :
+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   600|      1|   4.0|
|   600|     16|   4.0|
|   600|     32|   5.0|
|   600|     34|   5.0|
|   600|     47|   2.0|
|   600|     50|   5.0|
|   600|     70|   2.0|
|   600|    111|   1.0|
|   600|    150|   4.0|
|   600|    160|   2.0|
|   600|    177|   2.0|
|   600|    188|   3.0|
|   600|    196|   2.0|
|   600|    246|   4.0|
|   600|    247|   3.0|
|   600|    253|   3.0|
|   600|    260|   5.0|
|   600|    266|   4.0|
|   600|    273|   4.0|
|   600|    296|   1.0|
+------+-------+------+
only showing top 20 rows



In [15]:
# 13- Appliquer le modèle créé sur le user 600
# Recommander des films pour l'utilisateur 600
user_600_recommendations = model.recommendForUserSubset(
    spark.createDataFrame([(600,)], ["userId"]),
    10  # Nombre de recommandations
)

print("Recommandations pour l'utilisateur 600 :")
user_600_recommendations.show(truncate=False)

Recommandations pour l'utilisateur 600 :
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                              |
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|600   |[{102684, 11.079477}, {1859, 9.718053}, {26631, 9.040137}, {79318, 8.923627}, {64114, 8.393925}, {25905, 8.326016}, {94266, 8.267231}, {26122, 8.231591}, {1529, 8.141736}, {6896, 7.931106}]|
+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:



# Générer des prédictions pour tous les films non notés

# Créer un dataframe avec tous les movieIds uniques
all_movies = df_clean.select('movieId').distinct()

# Créer un dataframe avec l'utilisateur 600 pour chaque film
user_600_with_all_movies = all_movies.withColumn('userId', lit(600))

# Faire des prédictions
user_600_predictions = model.transform(user_600_with_all_movies)

# Exclure les films déjà notés par l'utilisateur 600
movies_rated_by_600 = user_600_ratings.select('movieId')
user_600_new_recommendations = user_600_predictions.join(
    movies_rated_by_600,
    user_600_predictions.movieId == movies_rated_by_600.movieId,
    "left_anti"
)

# Afficher les meilleures recommandations
print("Top 10 des films recommandés pour l'utilisateur 600 :")
user_600_new_recommendations.orderBy('prediction', ascending=False).show(10)



Top 10 des films recommandés pour l'utilisateur 600 :
+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
| 102684|   600| 11.079478|
|   1859|   600|  9.718052|
|  26631|   600|  9.040137|
|  79318|   600|  8.923628|
|  64114|   600|  8.393925|
|  25905|   600|  8.326017|
|  94266|   600|  8.267231|
|  26122|   600|  8.231591|
|   1529|   600|  8.141735|
|   6896|   600| 7.9311056|
+-------+------+----------+
only showing top 10 rows



In [17]:
# Arrêter la SparkSession
spark.stop()